In [1]:
# Importar a biblioteca re para expressões regulares
import re

## Atividade avaliativa 1


**Faça um programa que receba duas strings contendo equações lineares com duas incógnitas e resolva esse sistema linear. Indique no código qual método utilizou para resolver seu sistema linear. Sugestão: utilize expressões regulares.**

  
.  
  
  

 Observações:
  - As equações terão coeficientes inteiros.
  - Os coeficientes poderão ser 0, positivos ou negativos
  - Os coeficientes 1 e -1 não aparecerão explicitamente
  - Sistemas que não têm solução única deverão mostrar essa informação ao usuário
  - Entrega: 26/03/2025



In [2]:
# Exemplo de equações lineares que podem ser utilizadas (o seu programa vai receber duas delas)
'x - 2y = 8'
'-x + 5y = 80'
'2x = 9'
'4y = 12'
'-2x - 5y  =-1'

'-2x - 5y  =-1'

In [3]:
def parse_equation(equation):
    equation = equation.replace(' ', '')  # Remove espaços
    parts = equation.split('=')
    if len(parts) != 2:
        raise ValueError("Equação inválida: formato incorreto.")
    lhs, rhs = parts

    # Inicializa coeficientes
    a_x, a_y = 0, 0

    # Busca termos com x ou y
    terms = re.findall(r'([+-]?[\d]*[xy])', lhs)
    for term in terms:
        if term[-1] not in ('x', 'y'):
            raise ValueError(f"Termo inválido: {term}")
        var = term[-1]
        coeff = term[:-1] if term[:-1] else None

        # Processa coeficiente
        if coeff in (None, '+'):
            coeff_val = 1
        elif coeff == '-':
            coeff_val = -1
        else:
            coeff_val = int(coeff)

        if var == 'x':
            a_x += coeff_val
        else:
            a_y += coeff_val

    # Processa o lado direito
    try:
        rhs_val = int(rhs)
    except ValueError:
        raise ValueError(f"RHS inválido: {rhs}")

    return a_x, a_y, rhs_val


In [4]:
def solve_linear_system(eq1, eq2):
    try:
        a1, b1, c1 = parse_equation(eq1)
        a2, b2, c2 = parse_equation(eq2)
    except ValueError as e:
        return f"Erro: {e}"

    # Monta a matriz aumentada
    matrix = [
        [a1, b1, c1],
        [a2, b2, c2]
    ]

    # Passo 1: Verifica e troca linhas se necessário
    if matrix[0][0] == 0:
        if matrix[1][0] != 0:
            # Troca as linhas
            matrix[0], matrix[1] = matrix[1], matrix[0]
        else:
            # Ambas linhas têm coeficiente x = 0
            if matrix[0][1] == 0 and matrix[1][1] == 0:
                if matrix[0][2] != matrix[1][2]:
                    return "O sistema não tem solução única."
                else:
                    return "O sistema não tem solução única (infinitas soluções)."
            else:
                # Verifica se há solução para y
                if matrix[0][1] == 0:
                    if matrix[0][2] != 0:
                        return "O sistema não tem solução única."
                    else:
                        y = matrix[1][2] / matrix[1][1]
                        return f"O sistema não tem solução única (x pode ser qualquer valor, y = {y})."
                elif matrix[1][1] == 0:
                    if matrix[1][2] != 0:
                        return "O sistema não tem solução única."
                    else:
                        y = matrix[0][2] / matrix[0][1]
                        return f"O sistema não tem solução única (x pode ser qualquer valor, y = {y})."
                else:
                    # Verifica consistência entre as equações em y
                    y1 = matrix[0][2] / matrix[0][1]
                    y2 = matrix[1][2] / matrix[1][1]
                    if y1 != y2:
                        return "O sistema não tem solução única."
                    else:
                        return f"O sistema não tem solução única (x pode ser qualquer valor, y = {y1})."

    # Passo 2: Eliminação de Gauss
    a1, b1, c1 = matrix[0]
    a2, b2, c2 = matrix[1]

    # Calcula o fator para eliminar x da segunda linha
    fator = a2 / a1
    nova_linha2 = [
        a2 - fator * a1,          # Coeficiente de x (deve ser 0)
        b2 - fator * b1,          # Coeficiente de y
        c2 - fator * c1           # Termo constante
    ]

    # Atualiza a matriz
    matrix[1] = nova_linha2

    # Verifica se o sistema é impossível ou indeterminado
    if matrix[1][0] == 0 and matrix[1][1] == 0:
        if matrix[1][2] != 0:
            return "O sistema não tem solução única."
        else:
            return "O sistema não tem solução única (infinitas soluções)."

    # Passo 3: Resolve por substituição reversa
    y = matrix[1][2] / matrix[1][1]
    x = (matrix[0][2] - matrix[0][1] * y) / matrix[0][0]

    return (x, y)

In [5]:
# Exemplo de uso
eq1 = 'x - 2y = 8'
eq2 = '-x + 5y = 80'
print(solve_linear_system(eq1, eq2))  # Saída: (66.66666666666667, 29.333333333333336)

(66.66666666666666, 29.333333333333332)
